In [48]:
!pip install seqeval

  Preparing metadata (setup.py) ... one
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 24.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 MB 37.2 MB/s eta 0:00:0000:0100:01
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=6d28a1cef3d506e698f19a6e9922103fe74a9d786bcf741d357c7a3dd30a1fbd
  Stored in directory: /root/.cache/pip/wheels/5f/b8/73/0b2c1a76b701a677653dd79ece07cfabd7457989dbfbdcd8d7
Successfully built seqeval

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [8]:
import os
import torch
import transformers
import peft
import datasets
import evaluate
import time
import re
assert torch.cuda.is_available(), "you need cuda for this part"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


In [9]:
#device = 'cpu'

In [10]:
label_list = ["C","T"]
id2label = {
    0: "C",
    1: "T",
}

label2id = {
    "C": 0,
    "T": 1,
}
pattern = r'END\s+[A-Za-z0-9\_]+\.'

In [11]:
def remove_multiline_comments(text):
    # Регулярное выражение для поиска многострочных комментариев
    pattern = r'\(\*[^)]*?\n.*?\*\)'

    # Используем re.sub для замены всех найденных многострочных комментариев на пустую строку
    result = re.sub(pattern, '', text, flags=re.DOTALL)

    return result

# Пример использования
text = """
Это пример текста.
(* Это однострочный комментарий *)
Это продолжение текста.
(*
Это многострочный комментарий
на несколько строк
*)
Это еще текст.
"""

cleaned_text = remove_multiline_comments(text)
print(cleaned_text)


Это пример текста.
(* Это однострочный комментарий *)
Это продолжение текста.

Это еще текст.



In [12]:


code_dataset = datasets.load_dataset("hodza/BlackBox.Shkola.2014")['train']
#code_dataset = code_dataset.filter(lambda example: '/Mod/' in example["names"])

CODE_WORDS = [
    'ABSTRACT','EXTENSIBLE','POINTER',
	'ARRAY','FOR','PROCEDURE',
	'BEGIN','IF','RECORD',
	'BY','IMPORT','REPEAT',
	'CASE','IN','RETURN',
	'CLOSE','IS','THEN',
	'CONST','LIMITED','TO',
	'DIV','LOOP','TYPE',
	'DO','MOD','UNTIL',
	'ELSE','MODULE','VAR',
	'ELSIF','NIL','WHILE',
	'EMPTY','OF','WITH',
	'END','OR',
	'EXIT','OUT']

def is_contains_code(line):
    return any(line.find(s) != -1 for s in CODE_WORDS)

def strip_after_end(sample):
    text = re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f-\xff]', '', sample['texts'])
    text = remove_multiline_comments(text)
    lines = text.split('\n')
    if 'Docu/' in sample['names']:
        code = [l for l in lines if is_contains_code(l)]
        text = [l for l in lines if not is_contains_code(l)]
    elif 'Rsrc/' in sample['names']:
        code = []
        text = []
    else:
        code = []
        text = []
        is_code = True
        for l in lines:
            if is_code:
                code.append(l)
            else:
                if is_contains_code(l):
                    code.append(l)
                else:
                    text.append(l)
            match = re.search(pattern, l)
            if match:
                if not is_code:
                    code += text
                    text = []
                is_code = False
    return { "code_lines" : code,  "text_lines" : text, 'names' : sample['names']}
        
code_dataset = code_dataset.map(strip_after_end)
code_lines = [l for x in code_dataset for l in x['code_lines'] if not l.isspace() and len(l) > 0]
text_lines = [l for x in code_dataset for l in x['text_lines'] if not l.isspace() and len(l) > 0]


In [13]:
len(text_lines), len(code_lines)

(27422, 150620)

In [14]:
text_lines[:10]

['After compiling the module, a dialog box can be created for the items exported by ObxAddress1 using command NewForm... from menu Controls. Just enter the name ObxAddress1 into the Link field, and then click on the Create button. The type information extracted by the compiler is available to the BlackBox Component Builder at run-time, and is used to automatically create a data-entry form for the record declaration above. The form has a simple default layout. This default layout may be edited, and then opened as a dialog using the Open as Aux Dialog command in menu Controls.',
 'The text entry fields and the checkbox of the form are directly linked to the fields name, city, country, customer  and update of the record ObxAddress1.adr. The button is linked to the command OpenText, i.e., to the procedure exported by module ObxAddress1. Clicking the button causes procedure OpenText to be called. As a result, a new text is created; a textual report based on the variable adr is written to th

In [15]:
c = datasets.Dataset.from_dict({'text' : code_lines, 'label' : [0] * len(code_lines)})
t = datasets.Dataset.from_dict({'text' : text_lines, 'label' : [1] * len(text_lines)})

dataset = datasets.concatenate_datasets([c,t])
dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 178042
})

In [16]:
dataset[18419]

{'text': '\t\t\tCheckSym(rbrak);', 'label': 0}

In [17]:
dataset = dataset.train_test_split()

In [18]:
model_name = "distilbert/distilbert-base-cased"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)


In [19]:
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name, device_map=device, num_labels=2, id2label=id2label, label2id=label2id)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
data_collator = transformers.DataCollatorWithPadding(tokenizer=tokenizer)

In [21]:
import evaluate

accuracy = evaluate.load("accuracy")

In [22]:
import numpy as np
#labels = [label_list[i] for i in dataset[f"tags"]]
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [23]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [24]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map: 100%|██████████| 44511/44511 [00:01<00:00, 43629.25 examples/s]


In [25]:
training_args = transformers.TrainingArguments(
    output_dir="./results/code_model_line",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    gradient_accumulation_steps = 2,
    load_best_model_at_end=True,
    push_to_hub=False,
 
)

trainer = transformers.Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    
)

trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
classifier = transformers.pipeline('text-classification', model=model, tokenizer=tokenizer)

In [ ]:
val_dataset = datasets.load_dataset("hodza/BlackBox.Shkola.2014")['train']


In [ ]:
print(val_dataset[1]['texts'])

In [115]:
test = re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f-\xff]', '', val_dataset[1]['texts'])
lines = test.split('\n')
res = classifier(lines)
for i in range(len(lines)):
    print(res[i]['label'], lines[i])

C 
C Проект ИНФОРМАТИКА-21
C 
T Внедрение современных технологий и методов программирования 
T в общие базовые курсы программирования 
T (5-11 классы средних школ и 1-3 курсы университетов, 
T независимо от профиля и специализации)
C с целью создания единого пространства 
C для преподавания основ ИТ 
T как естественного продолжения 
T единой общей системы преподавания основ математики
C 
C Подробная информация о проекте: http://www.inr.ac.ru/~info21/
C 
C Для вопросов и замечаний: info21@inr.ac.ru
C 
T Благодарности. Российские версии среды обучения и разработки программ, носящие название Блэкбокс, основаны на системе программирования BlackBox Component Builder, созданной в Oberon microsystems, Inc., в развитие идей, впервые реализованных в Системе Оберон Никлауса Вирта и Юрга Гуткнехта (Niklaus Wirth, Jurg Gutknecht, ETH, Zurich, Switzerland). Со-архитектор Блэкбокса Клеменс Шиперски (Clemens Szyperski) работает в исследовательском подразделении корпорации Майкрософт (Microsoft Resear

In [26]:
classifier('с целью создания единого пространства')

NameError: name 'classifier' is not defined